In [1]:
import requests
import json
import pandas as pd
import hashlib # hash 함수용 sha256 사용할 라이브러리
import random

## 노드의 블록 정보 확인

In [2]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1708512046.6226194,
   'transactions': []}],
 'length': 1}

## transaction 입력하기

In [3]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "test_to",
        "amount": 3,
        "smart_contract": {"contract_address":"myaddress"}
}
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

b'{"message":"Transaction will be added to Block {2}"}\n'

## 노드의 블록정보 확인 - 2

In [4]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1708512046.6226194,
   'transactions': []}],
 'length': 1}

##  채굴하기 - 1

In [5]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

<Response [200]>


## 노드의 블록정보 확인 - 3

In [6]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1708512046.6226194,
   'transactions': []},
  {'index': 2,
   'nonce': -1155,
   'previous_hash': 'e4db71c4017a3be25ac686217e7befebd9136bb353e7a75344c311f0a51926f9',
   'timestamp': 1708512071.719713,
   'transactions': [{'amount': 3,
     'recipient': 'test_to',
     'sender': 'test_from',
     'timestamp': 1708512067.475464},
    {'amount': 0.1,
     'recipient': 'node_5000',
     'sender': 'master',
     'timestamp': 1708512071.719713}]}],
 'length': 2}

## transaction 입력하기 -2  / 채굴 / 블록정보 확인

In [ ]:
## transaction 입력하기 -2
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "test_to2",
    "amount": 30,
}
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

## transaction3 입력하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "test_to3",
    "amount": 300,
}
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

##  채굴하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

## 노드의 블록정보 확인 - 4
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)

In [ ]:
res.text

## Pandas 를 활용한 거래내역 조회

In [ ]:
status_json = json.loads(res.text)
status_json['chain']    
tx_amount_l = []
tx_sender_l = []
tx_reciv_l  = []
tx_time_l   = []

for chain_index in range(len(status_json['chain'])):
    chain_tx = status_json['chain'][chain_index]['transactions']
    for each_tx in range(len(chain_tx)):
        tx_amount_l.append(chain_tx[each_tx]['amount'])
        tx_sender_l.append(chain_tx[each_tx]['sender'])
        tx_reciv_l.append(chain_tx[each_tx]['recipient'])
        tx_time_l.append(chain_tx[each_tx]['timestamp'])

df_tx = pd.DataFrame()
df_tx['timestamp'] = tx_time_l  
df_tx['sender'] = tx_sender_l 
df_tx['recipient'] = tx_reciv_l
df_tx['amount'] = tx_amount_l   
df_tx

## 거래내역 기반 계정별 잔액 조회

In [ ]:
df_sended = pd.DataFrame(df_tx.groupby('sender')['amount'].sum()).reset_index()
df_sended.columns = ['user','sended_amount']
df_received= pd.DataFrame(df_tx.groupby('recipient')['amount'].sum()).reset_index()
df_received.columns = ['user','received_amount']
df_received

df_status = pd.merge(df_received,df_sended, on ='user', how=  'outer').fillna(0)
df_status['balance'] = df_status['received_amount']  - df_status['sended_amount']  
df_status